# Building a pipeline for ERA5 data download=>

## 🔹 CHUNK 2 — User controls

In [13]:
START_YEAR  = 2023
START_MONTH = 1

END_YEAR    = 2023
END_MONTH   = 3

BASE_DIR = r"D:\PHD IIT KGP\Codes @ VSCODE\ERA5 DATA Download"   # change only this


## 🔹 CHUNK 3 — Resume / crash-recovery logic

In [14]:
import os

def read_log(logfile):
    done = set()
    if os.path.exists(logfile):
        with open(logfile, "r") as f:
            for line in f:
                done.add(line.strip().split()[0])
    return done

def write_log(logfile, ym):
    with open(logfile, "a") as f:
        f.write(f"{ym} DONE\n")


## 🔹 CHUNK 4 — Required libraries

In [4]:
pip install cdsapi xarray netcdf4 tqdm numpy


Note: you may need to restart the kernel to use updated packages.


## 🔹 CHUNK 5 — ERA5 download logic

### 5.1 Month iterator

In [15]:
def month_range(start_year, start_month, end_year, end_month):
    months = []
    for y in range(start_year, end_year + 1):
        for m in range(1, 13):
            if (y == start_year and m < start_month):
                continue
            if (y == end_year and m > end_month):
                continue
            months.append((y, m))
    return months


### 5.2 ERA5 monthly download

In [16]:
import cdsapi
import calendar

def download_month(year, month, out_file):
    c = cdsapi.Client()

    days = [f"{d:02d}" for d in range(1, calendar.monthrange(year, month)[1] + 1)]
    hours = [f"{h:02d}:00" for h in range(24)]

    c.retrieve(
        "reanalysis-era5-single-levels",
        {
            "product_type": "reanalysis",
            "variable": [
                "total_precipitation",
                "snowfall",
            ],
            "year": str(year),
            "month": f"{month:02d}",
            "day": days,
            "time": hours,
            "format": "netcdf",
        },
        out_file
    )


### 5.3 ZIP detection & extraction

✔ CRITICAL FIX APPLIED HERE

In [17]:
import zipfile
import os

def handle_possible_zip(nc_path, extract_dir):
    if zipfile.is_zipfile(nc_path):
        print("⚠ ZIP archive detected (disguised as .nc). Extracting...")
        with zipfile.ZipFile(nc_path, 'r') as z:
            z.extractall(extract_dir)
        os.remove(nc_path)
        return True
    return False


## 🔹 CHUNK 6 — Progress & file size reporting

In [18]:
def sizeof_gb(filepath):
    return os.path.getsize(filepath) / (1024**3)


## 🔹 CHUNK 7 — Hourly → daily accumulation

### ✅ CORRECTED CHUNK 7 (FINAL VERSION)

In [19]:
# 🔹 CHUNK 7 — Hourly → Daily Accumulation (ERA5-safe)

import xarray as xr

def hourly_to_daily(infile, outfile):
    ds = xr.open_dataset(infile)

    # ---- FIX 1: normalize time coordinate ----
    if "time" not in ds.coords:
        if "valid_time" in ds.coords:
            ds = ds.rename({"valid_time": "time"})
        else:
            raise ValueError(
                "No recognizable time coordinate (time/valid_time) found in dataset."
            )

    # ---- FIX 2: daily accumulation ----
    ds_daily = ds.resample(time="1D").sum()

    # ---- FIX 3: ERA5 variable names & units ----
    if "tp" in ds_daily:
        ds_daily["tp"] *= 1000
        ds_daily["tp"].attrs["units"] = "mm/day"
        ds_daily["tp"].attrs["long_name"] = "Daily total precipitation"

    if "sf" in ds_daily:
        ds_daily["sf"] *= 1000
        ds_daily["sf"].attrs["units"] = "mm/day"
        ds_daily["sf"].attrs["long_name"] = "Daily snowfall"

    ds_daily.to_netcdf(outfile)
    ds.close()


## 🔹 CHUNK 8 — Main driver script

### ✅ CHUNK 8A — Disk growth monitor

In [22]:
# 🔹 CHUNK 8A — SAFE file verification & size reporting (Windows-safe)

import os
import time

def verify_download(filepath, wait=5):
    """
    Waits briefly after CDS finishes and verifies
    file existence and final size.
    """
    time.sleep(wait)

    if not os.path.exists(filepath):
        raise FileNotFoundError("Download finished but file not found.")

    size_gb = os.path.getsize(filepath) / (1024 ** 3)
    print(f"📦 Download verified. File size: {size_gb:.2f} GB")

    return size_gb


### 🔹 UPDATED CHUNK 8 — Main driver (with monitoring)

In [23]:
# 🔹 CHUNK 8 — Main driver script (Windows-safe, ZIP-safe)

import os
from tqdm import tqdm

RAW_DIR   = os.path.join(BASE_DIR, "raw_hourly")
DAILY_DIR = os.path.join(BASE_DIR, "daily_accumulated")
LOG_FILE  = os.path.join(BASE_DIR, "download_log.txt")

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(DAILY_DIR, exist_ok=True)

done_months = read_log(LOG_FILE)
months = month_range(START_YEAR, START_MONTH, END_YEAR, END_MONTH)

for year, month in tqdm(months, desc="ERA5 Monthly Download"):
    ym = f"{year}-{month:02d}"
    if ym in done_months:
        continue

    raw_file = os.path.join(RAW_DIR, f"tp_sf_{year}_{month:02d}.nc")

    print(f"\n⬇️ Downloading {ym} ...")
    download_month(year, month, raw_file)

    # ---- SAFE verification (NO file locking) ----
    verify_download(raw_file)

    # ---- ZIP-IN-DISGUISE HANDLING ----
    was_zip = handle_possible_zip(raw_file, RAW_DIR)
    if was_zip:
        raw_file = os.path.join(
            RAW_DIR, "data_stream-oper_stepType-accum.nc"
        )

    daily_file = os.path.join(
        DAILY_DIR, f"ERA5_DAILY_TP_SF_{year}_{month:02d}.nc"
    )

    print("🧮 Converting to daily accumulated...")
    hourly_to_daily(raw_file, daily_file)

    write_log(LOG_FILE, ym)


ERA5 Monthly Download:   0%|          | 0/3 [00:00<?, ?it/s]


⬇️ Downloading 2023-01 ...


2025-12-17 12:11:40,881 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-12-17 12:11:41,757 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)
2025-12-17 12:11:41,760 INFO Request ID is 0b5878bb-d81a-47c2-83ca-2b0133a5c254
2025-12-17 12:11:41,978 INFO status has been updated to accepted
2025-12-17 12:11:51,053 INFO status has been updated to successful


📦 Download verified. File size: 0.92 GB
🧮 Converting to daily accumulated...


ERA5 Monthly Download:  33%|███▎      | 1/3 [11:29<22:59, 689.98s/it]

...

ERA5 Monthly Download:  67%|██████▋   | 2/3 [21:27<10:35, 635.70s/it]


⬇️ Downloading 2023-03 ...


2025-12-17 12:33:08,519 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-12-17 12:33:09,400 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)
2025-12-17 12:33:09,400 INFO Request ID is 11653a3b-4fa0-4269-94f6-a65cffd327ce
2025-12-17 12:33:09,639 INFO status has been updated to accepted
2025-12-17 12:33:23,977 INFO status has been updated to running
2025-12-17 12:37:32,008 INFO status has been updated to successful
Recovering from connection erro

📦 Download verified. File size: 0.91 GB
🧮 Converting to daily accumulated...


ERA5 Monthly Download: 100%|██████████| 3/3 [32:19<00:00, 646.56s/it]
